# Scrapper a pagina web abierta con estadísticas ATP

## Descripción:
    
    La página web cuenta con el registro de las estadísticas globales de 54640 tenistas.
    
## Primero:
    
    Hago pruebas para acceder a un tenista cualquiera y me monto una lista con todas sus estadísticas.
    
## Segundo:
    
    Estudio peculiaridades de la página web: si cambio en la url el playerId = ... cambia a otro tenista.
    Además compruebo que el playerId = 0 no existe y que el último es playerId = 54640.
    Además, veo que hay Ids que no tienen estadísticas y otros donde solo hay 3. Esos los descartaré

## Finalmente:

    Me monto un bucle donde recorreré todos los Ids para montarme una lista con todas las estadísticas para todos los tenistas (los 54640)

In [ ]:
# pip install BeautifulSoup4
# pip install pandas
# pip install numpy
# pip install selenium
# pip instal webdriver_manager (este se usa para descargar automaticamente ChromeDriverManager en la misma versión que tu Google Chrome)
# y es necesario porque usamos todo el rato el ChromeDriver

In [ ]:
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import numpy as np
import time
import random

# Driver de selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # Para modificar las opciones de WebDriver en Chrome
from selenium.webdriver.chrome.service import Service

# Para instalar automáticamente el ChromeDriver correspondiente
from webdriver_manager.chrome import ChromeDriverManager

## Primero: Prueba con un único jugador y me monto sus stats

Escribimos la url de la web a scrappear

In [ ]:
url = "https://www.ultimatetennisstatistics.com/playerProfile?playerId=1000&tab=statistics"

Instalamos ChromeDriverManager. Webdriver_manager nos descarga la versión de nuestro actual Chrome porque deben coincidir. Nos devuelve la ruta donde lo guarda y lo almaceno en una variable 'ruta', nos servirá más adelante.

In [ ]:
ruta = ChromeDriverManager().install()

Definimos las opciones de nuestro navegador Chrome, que no son más que preferencias, aunque algunas importantes.

In [ ]:
options = Options()

Defino un user agent predeterminado, el mío. Quien haga uso de este cuaderno debe poner el suyo.
Para saber cual es el tuyo buscas en Google: 'my user agent' y te sale.

In [ ]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
options.add_argument(f"user-agent = {user_agent}") # Lo añado a mis preferencias

Se pueden añadir muchas más opciones (a gusto del consumidor), las que a mi me gustan son estas:

In [ ]:
# Deshabilitamos notificaciones.
options.add_argument("--disable-notifications")

# Para que no ejecute tareas que se ejecutan la primera vez que runeas Chrome
options.add_argument("--no-first-run")

# Evita que el servidor no detecte que somos un bot, esta es importante.
options.add_argument("--disable-blink-features=AutomationControlled")

Vamos a abrir nuestra web :)

In [ ]:
s = Service(ruta)
driver = webdriver.Chrome(service = s, options = options)
driver.get(url)

Ahora hago uso de Selenium para clickar en las cookies.

In [ ]:
driver.find_element("xpath", '//*[@class="fc-button fc-cta-consent fc-primary-button"]').click()

In [ ]:
driver.find_element("xpath", '//*[@class="btn btn-warning margin-left"]').click()

Ahora empezamos a hacer uso de BeautifulSoup para encontrar elementos del codigo html de la página.

In [ ]:
# Lo primero creo mi sopa
soup = BeautifulSoup(driver.page_source, "html.parser")

El atributo page_source es el html. Ahora mismo mi sopa es básicamente un string con el código fuente donde podemos encontrar elementos. Pues vamos a buscarlos con el método find_all().

Echando un ojo a la web, veo que las stats son elementos th y con esa con esa clase

In [ ]:
a = soup.find_all("th", class_ = "text-right pct-data")
a[:]

[<th class="text-right pct-data">2.4%</th>,
 <th class="text-right pct-data">2.8%</th>,
 <th class="text-right pct-data">58.9%</th>,
 <th class="text-right pct-data">63.2%</th>,
 <th class="text-right pct-data">45.0%</th>,
 <th class="text-right pct-data">51.0%</th>,
 <th class="text-right pct-data">55.7%</th>,
 <th class="text-right pct-data">63.5%</th>,
 <th class="text-right pct-data">5.2%</th>,
 <th class="text-right pct-data">4.8%</th>,
 <th class="text-right pct-data">29.3%</th>,
 <th class="text-right pct-data">50.1%</th>,
 <th class="text-right pct-data">38.7%</th>,
 <th class="text-right pct-data">37.3%</th>,
 <th class="text-right pct-data">20.7%</th>,
 <th class="text-right pct-data">46.3%</th>,
 <th class="text-right pct-data">44.8%</th>,
 <th class="text-right pct-data">36.1%</th>,
 <th class="text-right pct-data">
 <a href="/playerProfile?playerId=1000&amp;tab=matches&amp;season=&amp;fromDate=&amp;toDate=&amp;level=&amp;bestOf=&amp;surface=&amp;indoor=&amp;speed=&amp;roun

In [ ]:
len(a)

77

Aquí hay una ristra de elementos (77) pero que solo me interesan unos pocos de los primeros que son los de las estadísticas globales, los demás son de estadísticas filtradas por tipo de superficie, año, etc etc...

In [ ]:
stats = []
serve = a[0:8]

for element in serve:
    stats.append(element.text.replace('%',''))

stats

['2.4', '2.8', '58.9', '63.2', '45.0', '51.0', '55.7', '63.5']

In [ ]:
retur = a[8:15]

for element in retur:
    stats.append(element.text.replace('%',''))

stats

['2.4',
 '2.8',
 '58.9',
 '63.2',
 '45.0',
 '51.0',
 '55.7',
 '63.5',
 '5.2',
 '4.8',
 '29.3',
 '50.1',
 '38.7',
 '37.3',
 '20.7']

In [ ]:
total = a[15:18]

for element in total:
    stats.append(element.text.replace('%',''))

stats

['2.4',
 '2.8',
 '58.9',
 '63.2',
 '45.0',
 '51.0',
 '55.7',
 '63.5',
 '5.2',
 '4.8',
 '29.3',
 '50.1',
 '38.7',
 '37.3',
 '20.7',
 '46.3',
 '44.8',
 '36.1']

In [ ]:
victory_percentage = a[18]
victory_percentage = victory_percentage.text.replace('%','').replace('\n','')
stats.append(victory_percentage)

Ya debería estar mi lista de estadísticas para ese jugador (Novak Djokovic) montada, veamoslo:

In [ ]:
stats

['2.4',
 '2.8',
 '58.9',
 '63.2',
 '45.0',
 '51.0',
 '55.7',
 '63.5',
 '5.2',
 '4.8',
 '29.3',
 '50.1',
 '38.7',
 '37.3',
 '20.7',
 '46.3',
 '44.8',
 '36.1',
 '30.9']

Voy ahora a crearme las columnas de mi dataframe, acceder a los nombres de cada stat vaya.

In [ ]:
b = soup.find_all("td")
b

[<td>Ace %</td>,
 <td>Double Fault %</td>,
 <td>1st Serve %</td>,
 <td>1st Serve Won %</td>,
 <td>2nd Serve Won %</td>,
 <td>Break Points Saved %</td>,
 <td>Service Points Won %</td>,
 <td>Service Games Won %</td>,
 <td>Ace Against %</td>,
 <td>Double Fault Against %</td>,
 <td>1st Srv. Return Won %</td>,
 <td>2nd Srv. Return Won %</td>,
 <td>Break Points Won %</td>,
 <td>Return Points Won %</td>,
 <td>Return Games Won %</td>,
 <td title="Points Dominance Ratio: % of return points won divided by % of service points lost">Points Dominance</td>,
 <td title="Games Dominance Ratio: % of return games won divided by % of service games lost">Games Dominance</td>,
 <td title="Break Points Ratio: % of break points converted divided by % of faced break points lost">Break Points Ratio</td>,
 <td>Total Points Won %</td>,
 <td>Games Won %</td>,
 <td>Sets Won %</td>,
 <td>Matches Won %</td>,
 <td>Match Time</td>,
 <td>Aces</td>,
 <td>Ace %</td>,
 <td>Aces per Svc. Game</td>,
 <td>Aces per Set</td>,


Igual que antes me interesan solo unas pocas del principio

In [ ]:
columnas = []

for element in b[0:15]:
    columnas.append(element.text.replace('%',''))

for element in b[18:22]:
    columnas.append(element.text.replace('%',''))

In [ ]:
columnas

['Ace ',
 'Double Fault ',
 '1st Serve ',
 '1st Serve Won ',
 '2nd Serve Won ',
 'Break Points Saved ',
 'Service Points Won ',
 'Service Games Won ',
 'Ace Against ',
 'Double Fault Against ',
 '1st Srv. Return Won ',
 '2nd Srv. Return Won ',
 'Break Points Won ',
 'Return Points Won ',
 'Return Games Won ',
 'Total Points Won ',
 'Games Won ',
 'Sets Won ',
 'Matches Won ']

Ahora ya podriamos montarnos nuestro dataframe

In [ ]:
import pandas as pd

In [ ]:
tabla = pd.DataFrame(stats)

In [ ]:
tabla

,0
0,2.4
1,2.8
2,58.9
3,63.2
4,45.0
5,51.0
6,55.7
7,63.5
8,5.2
9,4.8


In [ ]:
driver.quit()

## Segunda: ¿Qué pasa con jugadores sin stats o solo 3 stats?

Si cambiamos la url al playerId = 1, nos sale una página con un jugador que solo tiene 3 stats

In [ ]:
url = "https://www.ultimatetennisstatistics.com/playerProfile?playerId=1&tab=statistics"
ruta = ChromeDriverManager().install()
options = Options()

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
options.add_argument(f"user-agent = {user_agent}")
options.add_argument("--disable-notifications")
options.add_argument("--no-first-run")
options.add_argument("--disable-blink-features=AutomationControlled")

s = Service(ruta)
driver = webdriver.Chrome(service = s, options = options)
driver.get(url)

In [ ]:
driver.find_element("xpath", '//*[@class="fc-button fc-cta-consent fc-primary-button"]').click()

In [ ]:
driver.find_element("xpath", '//*[@class="btn btn-warning margin-left"]').click()

In [ ]:
soup = BeautifulSoup(driver.page_source, "html.parser")
a = soup.find_all("th", class_ = "text-right pct-data") # ¿por qué busco este elemento?
a[:]

[<th class="text-right pct-data">32.5%</th>,
 <th class="text-right pct-data">11.1%</th>,
 <th class="text-right pct-data">
 <a href="/playerProfile?playerId=1&amp;tab=matches&amp;season=&amp;fromDate=&amp;toDate=&amp;level=&amp;bestOf=&amp;surface=&amp;indoor=&amp;speed=&amp;round=&amp;result=&amp;tournamentId=&amp;opponent=&amp;countryId=&amp;outcome=played" title="Show matches">0.0%</a>
 </th>,
 <th class="text-right pct-data">32.5%</th>,
 <th class="text-right pct-data">8.56</th>,
 <th class="text-right pct-data">19.3</th>,
 <th class="text-right pct-data">0.0%</th>,
 <th class="text-right pct-data">0.0%</th>,
 <th class="text-right pct-data">0.00</th>,
 <th class="text-right pct-data">11.1%</th>,
 <th class="text-right pct-data">2.25</th>,
 <th class="text-right pct-data">
 <a href="/playerProfile?playerId=1&amp;tab=matches&amp;season=&amp;fromDate=&amp;toDate=&amp;level=&amp;bestOf=&amp;surface=&amp;indoor=&amp;speed=&amp;round=&amp;result=&amp;tournamentId=&amp;opponent=&amp;cou

In [ ]:
len(a)

15

Cuando el jugador solo tiene 3 estadísticas solo hay 15 elementos en 'a'. Este caso lo vamos a descartar cuando armemos el dataset porque me interesan solo los jugadores con las estadísticas al completo, es decir cuando 'a' tiene al menos 77 elementos.

In [ ]:
driver.quit()

## Último: Voy a montar el dataset

Cambiar el id_inicial y id_final si se quiere hacer por paquetes. De golpe sería recorrer del 0 al 54640. Lo mejor es ir haciéndolo de poco en poco, de 2500 en 2500 está bien.

In [ ]:
# Configuración de ChromeDriver
ruta = ChromeDriverManager().install()
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--no-first-run")
options.add_argument("--disable-blink-features=AutomationControlled")
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")
s = Service(ruta)
driver = webdriver.Chrome(service = s, options = options)

# Rango de IDs para el paquete actual
id_inicial = 1
id_final = id_inicial + 54639
csv_file = 'tennis_players_historical_and_individual_stats.csv'

columnas = [
    "Ace", "Double Fault", "1st Serve", "1st Serve Won", "2nd Serve Won",
    "Break Points Saved", "Service Points Won", "Service Games Won",
    "Ace Against", "Double Fault Against", "1st Serve Return Won", "2nd Serve Return Won",
    "Break Points Won", "Return Points Won", "Return Games Won", "Total Points Won",
    "Games Won", "Sets Won", "Matches Won"
]

# Crear un archivo CSV vacío con las columnas correctas si no existe
if not pd.io.common.file_exists(csv_file):
    pd.DataFrame(columns = columnas).to_csv(csv_file, index = False)

# Función para añadir filas al archivo CSV
def append_to_csv(data, filename):
    df = pd.DataFrame(data, columns = columnas)
    df.to_csv(filename, mode = 'a', header = False, index = False)

# Lista para almacenar las estadísticas de todos los jugadores en el paquete actual
all_players_stats = []

# Itero sobre los IDs en el rango actual
for player_id in range(id_inicial, min(id_final + 1, 54641)):
    print(player_id)
    url = f'https://www.ultimatetennisstatistics.com/playerProfile?playerId={player_id}&tab=statistics'
    driver.get(url)
    time.sleep(3)
    try:
        driver.find_element("xpath", '//*[@class="fc-button fc-cta-consent fc-primary-button"]').click()
        driver.find_element("xpath", '//*[@class="btn btn-warning margin-left"]').click()
    except:
        pass

    soup = BeautifulSoup(driver.page_source, "html.parser")
    a = soup.find_all("th", class_ = "text-right pct-data")

    if not a or len(a) < 77:
        continue

    stats = [element.text.replace('%', '').replace('\n', '') for element in a[:19]]
    all_players_stats.append(stats)

    # Escribo los resultados en el archivo CSV después de cada jugador
    append_to_csv([stats], csv_file)

In [ ]:
driver.quit()